In [1]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
# ========== python ==========
import os
from pathlib import Path
from logging import Logger
from typing import List, Dict, Tuple, Optional, Union, Callable, Final, Literal, get_args
from operator import itemgetter, attrgetter
import itertools
from IPython.display import display

from utils.setup import setup_logger, get_device
from const.const_values import PROJECT_DIR

os.chdir(PROJECT_DIR)
logger: Logger = setup_logger(__name__, f'{PROJECT_DIR}/log/jupyter_run.log')
device = get_device(device_name='cpu', logger=logger)

In [2]:
# jupyter
import seaborn as sns
import matplotlib.pyplot as plt
# Machine learning
import numpy as np
import pandas as pd
import h5py
import optuna
# torch
import torch
from torch.nn import CrossEntropyLoss
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader
from torch.utils.tensorboard.writer import SummaryWriter
# torch ignite
from ignite.engine import Engine
from ignite.handlers import Checkpoint
# My items
from models.datasets.data_helper import MyDataHelper, MyDataLoaderHelper, DefaultTokens
from models.datasets.datasets_for_story import StoryTriple
# My utils
from utils.setup import load_param
from utils.torch import load_model, torch_fix_seed
# main function
from run_for_KGC import main_function


In [3]:
from const.const_values import CPU, MODEL
from models.KGModel.kg_model import HEAD, RELATION, TAIL
from utils.torch_ignite import TRAINER, EVALUATOR
from const.const_values import DATASETS, DATA_HELPER, DATA_LOADERS, TRAIN_RETURNS

In [4]:
SEED: Final[int] = 42
args_path = f'{PROJECT_DIR}/models/kgc03/ResidentPatient075/param.pkl'
model_path = f'{PROJECT_DIR}/models/kgc03/ResidentPatient075/model.pth'

In [5]:
args = load_param(args_path)

# args.pre_train = True
args.logger = logger
args.device = device
args.batch_size = 1
args.pre_train=False
args.init_embedding_using_bert = False
args.model_path = model_path
args.only_load_trainer_evaluator = True

del args.optuna_file, args.device_name, args.pid, args.study_name, args.n_trials
logger.info(args)

2023-02-04 06:50:01 - INFO - 2517384385.py - 13 - Namespace(notebook=False, console_level='info', logfile='models/kgc03/ResidentPatient075/log.log', param_file='models/kgc03/ResidentPatient075/param.pkl', train_anyway=False, old_data=0, tensorboard_dir='models/kgc03/ResidentPatient075/tensorboard', checkpoint_dir='models/kgc03/ResidentPatient075/checkpoint/', model_path='/Users/ryoyakaneda/Documents/学校/M1Study/knowledge_graph/models/kgc03/ResidentPatient075/model.pth', resume_from_checkpoint=False, resume_from_last_point=False, only_load_trainer_evaluator=True, resume_checkpoint_path=None, pre_train=False, train_valid_test=False, only_train=True, use_for_challenge100=False, use_for_challenge090=False, use_for_challenge075=True, use_title='ResidentPatient', do_optuna=False, story_special_num=5, relation_special_num=5, entity_special_num=5, padding_token_e=0, cls_token_e=1, mask_token_e=2, sep_token_e=3, bos_token_e=4, padding_token_r=0, cls_token_r=1, mask_token_r=2, sep_token_r=3, bos_

In [6]:
torch_fix_seed(seed=SEED)
return_dict = main_function(args, logger=logger)

2023-02-04 06:50:01 - INFO - run_for_KGC.py - 893 - ----- make datahelper start. -----
2023-02-04 06:50:01 - INFO - data_helper.py - 333 - ====== Show MyRawData ======
2023-02-04 06:50:01 - INFO - data_helper.py - 334 - entity num: 7812
2023-02-04 06:50:01 - INFO - data_helper.py - 335 - relation num: 62
2023-02-04 06:50:01 - INFO - data_helper.py - 336 - ====== Show MyRawData ======
2023-02-04 06:50:01 - INFO - data_helper.py - 607 - ========== Show DataHelper ==========
2023-02-04 06:50:01 - INFO - data_helper.py - 333 - ====== Show MyRawData ======
2023-02-04 06:50:01 - INFO - data_helper.py - 334 - entity num: 7812
2023-02-04 06:50:01 - INFO - data_helper.py - 335 - relation num: 62
2023-02-04 06:50:01 - INFO - data_helper.py - 336 - ====== Show MyRawData ======
2023-02-04 06:50:01 - INFO - data_helper.py - 609 - entity_special_dicts: {0: '<pad_e>', 1: '<cls_e>', 2: '<mask_e>', 3: '<sep_e>', 4: '<bos_e>'}
2023-02-04 06:50:01 - INFO - data_helper.py - 610 - relation_special_dicts: {

In [7]:
model = return_dict[MODEL]

dataset_train: StoryTriple = return_dict[DATASETS][0]
triple: torch.Tensor = dataset_train.triple
data_helper: MyDataHelper = return_dict[DATA_HELPER]
evaluator: Checkpoint = return_dict[TRAIN_RETURNS][EVALUATOR]

load_model(model, args.model_path, device)
model.eval()

entities, relations = data_helper.processed_entities, data_helper.processed_relations
d_e, d_r = {e: i for i, e in enumerate(entities)}, {r: i for i, r in enumerate(relations)}

triple_df = pd.DataFrame([(entities[_t[0]], relations[_t[1]], entities[_t[2]]) for _t in triple], columns=[HEAD, RELATION, TAIL])
story_entities = triple_df[HEAD].tolist()

In [8]:
triple_df

,head,relation,tail
0,<bos_e>,<bos_r>,<bos_e>
1,AbbeyGrange:001,kgc:subject,AllTitle:Holmes
2,AbbeyGrange:001,kgc:hasPredicate,word.predicate:wakeUp
3,AbbeyGrange:001,kgc:whom,AllTitle:Watson
4,AbbeyGrange:001,kgc:infoSource,AllTitle:Watson
...,...,...,...
17469,AbbeyGrange:084,kgc:subject,AbbeyGrange:Theresa
17470,AbbeyGrange:084,kgc:hasPredicate,word.predicate:rest
17471,AbbeyGrange:084,kgc:where,AbbeyGrange:room_of_Lady_Brackenstall
17472,<bos_e>,<bos_r>,<bos_e>


In [9]:
def extract(_model, target, inputs):
    features = None

    def forward_hook(_module, _inputs, _):
        nonlocal features
        x, _, _ = _inputs
        outputs = _module.forward(x, x, x, need_weights=True)[1]
        features = outputs.detach().clone()

    handle = target.register_forward_hook(forward_hook)

    _model.eval()
    _model(inputs, torch.LongTensor([[]]), torch.LongTensor([[]]), torch.LongTensor([[]]))

    handle.remove()

    return features

def get_attention(input_):
    assert len(input_) == 1
    features = extract(model, model.transformer.layers[-1].self_attn, input_)[0]
    df_attention = pd.DataFrame([[entities[h], relations[r], entities[t]]+[features[j, i].item() for j in range(len(features))] for i, (h, r, t) in enumerate(input_[0])])
    df_attention.columns=[HEAD, RELATION, TAIL] + [f'atten_from{i}' for i in range(len(df_attention.columns)-3)]
    return df_attention

def show_attension_heatmap(df_attention):
    sns.heatmap(df_attention.iloc[:,3:])
    plt.show()

In [10]:
MASK_E = DefaultTokens.MASK_E
KILL = 'word.predicate:kill'

TAKE = 'word.predicate:take'
BRING = 'word.predicate:bring'
DIE = 'word.predicate:die'
HIDE = 'word.predicate:hide'

In [11]:
bos_triple = [d_e[DefaultTokens.BOS_E], d_r[DefaultTokens.BOS_R],d_e[DefaultTokens.BOS_E]]
mask_e_id = d_e[DefaultTokens.MASK_E]
Holmes_id =d_e['AllTitle:Holmes']

def make_ranking(from_story_name, to_story_name, predicate_, whom_, subject_, why_, what_, where_):
    if not (from_story_name is None and to_story_name is None):
        _start_index = story_entities.index(from_story_name)-1 
        _end_index = len(story_entities) - story_entities[::-1].index(to_story_name)
    else: 
        _start_index = 0
        _end_index = 0
    question_ = torch.tensor(
        [
            bos_triple, 
            [mask_e_id, d_r['kgc:infoSource'],     Holmes_id      ],
            [mask_e_id, d_r['kgc:hasPredicate'],   d_e[predicate_]],
            [mask_e_id, d_r['kgc:whom'],           d_e[whom_     ]],
            [mask_e_id, d_r['kgc:subject'],        d_e[subject_  ]],
            [mask_e_id, d_r['kgc:why'],            d_e[why_      ]],
            [mask_e_id, d_r['kgc:what'],           d_e[what_     ]],
            [mask_e_id, d_r['kgc:where'],          d_e[where_    ]],
        ]
    )
    mask_ = torch.zeros_like(question_, dtype=torch.bool) # not mask all position
    mask_[1:, 0] = True                                   # where head position without bos token
    mask_[1:, 2] = True                                   # where tail position without bos token

    last_triples = triple[_start_index: _end_index]

    questions = torch.cat([last_triples, question_], dim=0).unsqueeze(0)
    masks = torch.cat([torch.zeros_like(last_triples), mask_], dim=0).to(torch.bool).transpose(1,0).unsqueeze(0)

    data_list = []
    with torch.no_grad():
        _, (story_pred, relation_pred, entity_pred) = model(questions, masks[:,0], masks[:,1], masks[:,2])
        sorted_ = torch.argsort(entity_pred, dim=1, descending=True)
        for i in range(sorted_.shape[1]):
            ans_= sorted_[:, i]
            info_source_, predicate_pred, whom_pred, subject_pred, why_pred, what_pred, where_pred = ans_
            data_list.append([entities[predicate_pred], entities[whom_pred], entities[subject_pred], entities[why_pred], entities[what_pred], entities[where_pred]])
    df_ranking = pd.DataFrame(data_list, columns=['predicate', 'whom', 'subject', 'why', 'what', 'where'])
    df_attension = get_attention(questions)
    
    return df_ranking, df_attension

In [16]:
def main_func01(_title, _victim_name, criminal, predicate, _last_index, _story_len):
    from_ = f'{_title}:{_last_index-_story_len+1}'
    to_ = f'{_title}:{_last_index}'
    predicate = predicate
    victim = f'{_title}:{_victim_name}'
    criminal = f'{_title}:{criminal}'
    df_ranking, df_attension = make_ranking(
        from_, to_, predicate, victim, MASK_E, MASK_E, MASK_E, MASK_E)

    pred_rank = df_ranking.index[df_ranking['subject']==criminal].tolist()
    pred_rank = pred_rank[0] if len(pred_rank)==1 else -1
    logger.info(f"The pred ranking about {criminal} is {pred_rank}")
    display(df_ranking.iloc[:max(50, pred_rank)])
    len_ = len(df_attension)
    for i in range(len_-10, len_):
        print(f"index={i}, triple={df_attension.iloc[i,:3].tolist()}, attention list")
        display(df_attension.sort_values(f'atten_from{i}', ascending=False).iloc[:,[0,1,2,3+i]],)
    return df_ranking, df_attension

def check_killer(_title, _victim_name, _killer_name, _last_index, _story_len):
    return main_func01(_title, _victim_name, _killer_name, KILL, _last_index, _story_len)

In [17]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)

### 入院患者
Who killed Blessington? (criminal & explanation)
被害者: Blessington
犯人: 3人
犯行動機:

In [ ]:
def do_ResidentPatient_pred():
    title = 'ResidentPatient'
    victim_name = 'Blessington'
    killer_name = 'Helper_boy'
    last_index = 243
    story_len = 80

    df_ranking, df_attention = check_killer(title, victim_name, killer_name, last_index, story_len)
    return df_ranking, df_attention

do_ResidentPatient_pred()

2023-02-04 06:52:30 - INFO - 3976068145.py - 12 - The pred ranking about ResidentPatient:Helper_boy is 198


,predicate,whom,subject,why,what,where
0,word.predicate:find,ResidentPatient:Blessington,ResidentPatient:Blessington,word.predicate:see,SilverBlaze:Silver_Blaze,ResidentPatient:Blessington_room
1,word.predicate:enter,SilverBlaze:Silver_Blaze,SilverBlaze:Silver_Blaze,SpeckledBand:corridor,ResidentPatient:Blessington,word.predicate:see
2,word.predicate:meet,SilverBlaze:Wife_of_John_Straker,AllTitle:Holmes,SilverBlaze:Edith_Baxter,ResidentPatient:Disgust,SilverBlaze:Edith_Baxter
3,word.predicate:go,ResidentPatient:Percy_Trevelyan,ResidentPatient:Young_man,SpeckledBand:money,SpeckledBand:1883-04-01T16,ACaseOfIdentity:Windibank
4,word.predicate:notExist,AllTitle:Holmes,SilverBlaze:Wife_of_John_Straker,ResidentPatient:Blessington_room,AllTitle:Watson,SpeckledBand:Julia_s_bedroom
5,word.predicate:notKnow,ResidentPatient:Young_man,AllTitle:Watson,SilverBlaze:John_Straker,DancingMen:Chicago,DateTime:1883-04-01T15:00:00
6,word.predicate:say,SilverBlaze:Silas_Brown,SpeckledBand:1883-04-01T17,SpeckledBand:Julia_s_bedroom,DevilsFoot:Case_1,CrookedMan:Nancy
7,word.predicate:see,<bos_e>,AbbeyGrange:Stanley_Hopkins,DateTime:1898-08-13T11:00:00,DancingMen:Dancing_dolls,SpeckledBand:the_day_Julia_died
8,word.predicate:kill,AllTitle:Watson,ResidentPatient:Elderly_man,SilverBlaze:Wife_of_John_Straker,SilverBlaze:Horseshoe_marks,word.predicate:go
9,word.predicate:live,SpeckledBand:1883-04-01T17,SpeckledBand:Julia,ResidentPatient:Elderly_man,ResidentPatient:Young_man,word.predicate:examine


index=390, triple=['ResidentPatient:243', 'kgc:subject', 'ResidentPatient:Cigar'], attention list


,head,relation,tail,atten_from390
390,ResidentPatient:243,kgc:subject,ResidentPatient:Cigar,0.023989
344,ResidentPatient:235,kgc:subject,ResidentPatient:Lanner_Inspector,0.015074
386,ResidentPatient:242,kgc:what,ResidentPatient:253,0.012249
316,ResidentPatient:230,kgc:subject,ResidentPatient:Blessington,0.011309
385,ResidentPatient:242,kgc:what,ResidentPatient:252,0.010519
387,ResidentPatient:242,kgc:what,ResidentPatient:254,0.010140
379,ResidentPatient:242,kgc:what,ResidentPatient:246,0.009974
123,ResidentPatient:189,kgc:subject,ResidentPatient:Blessington,0.009913
399,<mask_e>,kgc:where,<mask_e>,0.009763
311,ResidentPatient:229,kgc:subject,ResidentPatient:Blessington,0.009628


index=391, triple=['ResidentPatient:243', 'kgc:hasProperty', 'word.predicate:Habana_flavored'], attention list


...

index=398, triple=['<mask_e>', 'kgc:what', '<mask_e>'], attention list


,head,relation,tail,atten_from398
399,<mask_e>,kgc:where,<mask_e>,0.015324
390,ResidentPatient:243,kgc:subject,ResidentPatient:Cigar,0.014760
386,ResidentPatient:242,kgc:what,ResidentPatient:253,0.011932
387,ResidentPatient:242,kgc:what,ResidentPatient:254,0.011777
385,ResidentPatient:242,kgc:what,ResidentPatient:252,0.010534
376,ResidentPatient:242,kgc:what,ResidentPatient:243,0.010520
316,ResidentPatient:230,kgc:subject,ResidentPatient:Blessington,0.010211
396,<mask_e>,kgc:subject,<mask_e>,0.010189
378,ResidentPatient:242,kgc:what,ResidentPatient:245,0.009951
358,ResidentPatient:238,kgc:subject,ResidentPatient:Cigar_Pipes,0.009688


index=399, triple=['<mask_e>', 'kgc:where', '<mask_e>'], attention list


,head,relation,tail,atten_from399
399,<mask_e>,kgc:where,<mask_e>,0.026358
390,ResidentPatient:243,kgc:subject,ResidentPatient:Cigar,0.023184
344,ResidentPatient:235,kgc:subject,ResidentPatient:Lanner_Inspector,0.014394
316,ResidentPatient:230,kgc:subject,ResidentPatient:Blessington,0.013004
366,ResidentPatient:240,kgc:subject,ResidentPatient:One_of_the_cigar,0.012837
330,ResidentPatient:233,kgc:subject,ResidentPatient:Blessington,0.011808
311,ResidentPatient:229,kgc:subject,ResidentPatient:Blessington,0.011472
350,ResidentPatient:236,kgc:subject,ResidentPatient:Blessington,0.011274
396,<mask_e>,kgc:subject,<mask_e>,0.010832
336,ResidentPatient:234,kgc:subject,ResidentPatient:Lanner_Inspector,0.010292


(                       predicate                                    whom  \
 0            word.predicate:find             ResidentPatient:Blessington   
 1           word.predicate:enter                SilverBlaze:Silver_Blaze   
 2            word.predicate:meet        SilverBlaze:Wife_of_John_Straker   
 3              word.predicate:go         ResidentPatient:Percy_Trevelyan   
 4        word.predicate:notExist                         AllTitle:Holmes   
 ...                          ...                                     ...   
 7812         ResidentPatient:276  ACaseOfIdentity:Left_wrist_double_wire   
 7813  AbbeyGrange:French_windows                 ACaseOfIdentity:Quickly   
 7814            SpeckledBand:321                         DevilsFoot:Room   
 7815             AbbeyGrange:390                    ResidentPatient:168a   
 7816         ResidentPatient:293                     ACaseOfIdentity:558   
 
                                subject                               why 